### Data Cleaning

In [10]:
import pandas as pd

In [9]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))

import data_processing_functions as dpf

In [8]:
lego_sets = pd.read_csv('../data/raw/lego_sets.csv')

In [ ]:
dpf.show_basic_info(lego_sets)